In [1]:
from pyspark.sql import SparkSession



In [2]:
from pyspark import SparkContext
from  pyspark.sql import SQLContext


In [3]:
spark = SparkSession.builder \
             .master("local") \
             .appName("Word Count") \
             .config("spark.some.config.option", "some-value") \
            .getOrCreate()


In [5]:
import os


In [7]:
main_dir = '/home/henry/projects/Spark-The-Definitive-Guide/'
path = os.path.join(main_dir, 'data', 'activity-data_small')
os.path.isdir(path)

True

In [11]:
df = spark.read.json(path)
dataSchema = df.schema
dataSchema

StructType(List(StructField(Arrival_Time,LongType,true),StructField(Creation_Time,LongType,true),StructField(Device,StringType,true),StructField(Index,LongType,true),StructField(Model,StringType,true),StructField(User,StringType,true),StructField(gt,StringType,true),StructField(x,DoubleType,true),StructField(y,DoubleType,true),StructField(z,DoubleType,true)))

In [12]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).json(path)

In [15]:
activityCounts = streaming.groupBy("gt").count()

In [16]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [17]:
activityQuery = activityCounts.writeStream.queryName("activty_counts").format("memory").outputMode("complete").start()

In [ ]:
activityQuery.awaitTermination()

In [18]:
spark.streams.active

In [19]:
from time import sleep
for x in range(5):
    spark.sql("select * from activty_counts").show()
    sleep(1)

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|61546|
|     stand|56925|
|stairsdown|46827|
|      walk|66280|
|  stairsup|52258|
|      null|52239|
|      bike|53985|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|61546|
|     stand|56925|
|stairsdown|46827|
|      walk|66280|
|  stairsup|52258|
|      null|52239|
|      bike|53985|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|61546|
|     stand|56925|
|stairsdown|46827|
|      walk|66280|
|  stairsup|52258|
|      null|52239|
|      bike|53985|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|61546|
|     stand|56925|
|stairsdown|46827|
|      walk|66280|
|  stairsup|52258|
|      null|52239|
|      bike|53985|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|61546|
|     stand|56925|
|stairsdown|46827|
|      walk|66280|
|  stairsup|52258|
|      n

In [20]:
spark.sql("select * from activty_counts").show()


+----------+-----+
|        gt|count|
+----------+-----+
|       sit|73854|
|     stand|68310|
|stairsdown|56192|
|      walk|79536|
|  stairsup|62710|
|      null|62687|
|      bike|64783|
+----------+-----+

